Usage of the package
========

First, some imports

In [1]:
import gzip
import pickle
import random
from collections import namedtuple

from territories import Territory, MissingTreeCache

## Creation of the tree

The first step is to create a tree of known entities. This can be a very compute intensive task, depending on the tree size. That is why, by default, once created, the tree is stored on disk.

Here we will create a very simple tree out of the **tree.txt** file.

In [2]:
Node = namedtuple('Node', ('id', 'parent_id', 'label', 'level'))
split = lambda x: (arg if arg != 'null' else None for arg in x[:-1].split('; '))

try:
    Territory.load_tree()
except MissingTreeCache:
    with open("tree.txt", "r") as file:
        lines = file.readlines()
        stream = ([Node(*split(x) )for x in lines])
        Territory.build_tree(data_stream=stream, save_tree=False)

Then, you can start to create territories from arbitrary territoiral units.

Entities associated territories are represented in an efficient way : if all leaves of a parent node are included in the territory, they are simply replaced by their parent node.

In [3]:
# some node of the tree
print('\n'.join([f"{e.name} | {e.partition_type}" for e in random.sample(Territory.tree.nodes(), 12)]))

Monceaux-l'Abbaye | COM
Val-Mont | COM
Pila-Canale | COM
Chambry | COM
Tresques | COM
Doudrac | COM
Ners | COM
Bonneval-sur-Arc | COM
Sainte-Eulalie-en-Royans | COM
Guinecourt | COM
Auchy-lez-Orchies | COM
Thoirette-Coisia | COM


In [4]:
with open("tree_large.gzip", "rb") as file:
    lines = pickle.loads(gzip.decompress(file.read()))

stream = ([Node(*split(x) )for x in lines])
Territory.build_tree(data_stream=stream)

a = Territory.from_names("COM:69123", "COM:93055", "COM:94052")
b = Territory.from_names("COM:27429", "REG:84", "DEP:75")
c = Territory.from_names("COM:38185", "COM:31555", "REG:11")
d = Territory.from_names("COM:33063", "COM:13055", "REG:28")
e = Territory.from_names("COM:35238", "COM:35047", "DEP:27")
f = Territory.from_names("COM:59350", "COM:38442", "REG:53")

You can crate a territory with names of territorial units.

In [4]:
ter = Territory.from_names("DEP:69", "COM:59350", "ARR:75106")
ter

Paris 6e|Lille|Rhône

If names are invalid, an `NotOnTree` exception will be raised.

In [5]:
try:
    Territory.from_names("DEP:69", "do not exist", "garbage")
except Exception as e:
    print(e)

garbage, do not exist where not found in the territorial tree


Territories are jsons serializable, you can simply return them from an API endpoint

In [6]:
import json

print(json.dumps(ter, indent=4))

[
    {
        "name": "Rh\u00f4ne",
        "tu_id": "DEP:69",
        "atomic": false,
        "partition_type": "DEP",
        "postal_code": null
    },
    {
        "name": "Lille",
        "tu_id": "COM:59350",
        "atomic": true,
        "partition_type": "COM",
        "postal_code": null
    },
    {
        "name": "Paris 6e",
        "tu_id": "ARR:75106",
        "atomic": true,
        "partition_type": "ARR",
        "postal_code": null
    }
]


## Operations on territories


Usual operation on territories works as expected :

In [8]:
# addition

print(a, c)
print(a + c)

Pantin|Lyon|Nogent-sur-Marne Grenoble|Toulouse|Île-de-France
Grenoble|Lyon|Toulouse|Île-de-France


In [9]:
# substraction

print(a, d)
print(a - d)

Pantin|Lyon|Nogent-sur-Marne Bordeaux|Marseille|Normandie
Pantin|Lyon|Nogent-sur-Marne


More importantly, sets operations are also supported :

In [10]:
# intersection
print(f"Intersection of {a} and {d} is {a & d}")

# union
print(f"Union of {c} and {f} is {f | c}")

Intersection of Pantin|Lyon|Nogent-sur-Marne and Bordeaux|Marseille|Normandie is {}
Union of Grenoble|Toulouse|Île-de-France and Saint-Pierre-de-Chartreuse|Lille|Bretagne is Toulouse|Grenoble|Saint-Pierre-de-Chartreuse|Lille|Île-de-France|Bretagne


Territorial units may have parents or children, but Territory do not. As a territory may be formed of several territorial units, it has a LCA, a Lowest Common Ancestor.

In [11]:
lyon_and_grenoble = Territory.from_names("COM:38185", "COM:69123")

lyon_and_grenoble.lowest_common_ancestor()

Auvergne-Rhône-Alpes

You can easily retrieve all ancestors of a territory with the `.ancestors()` method, and respectively all of its descendants with the `.descendants()` method :

In [12]:
a.ancestors()

{France,
 Auvergne-Rhône-Alpes,
 Île-de-France,
 Val-de-Marne,
 Seine-Saint-Denis,
 Rhône}

In [13]:
print(a)
a.descendants(include_itself=True)

Pantin|Lyon|Nogent-sur-Marne


{Lyon,
 Pantin,
 Nogent-sur-Marne,
 Lyon 1er,
 Lyon 4e,
 Lyon 5e,
 Lyon 3e,
 Lyon 6e,
 Lyon 7e,
 Lyon 9e,
 Lyon 8e,
 Lyon 2e}

Territories are `True` if they are not empty, but you should probably use the `is_empty()` method for clarity.

In [14]:
if Territory.from_names("DEP:69"):
    print("not empty")

if Territory.from_names():
    print("empty")

if Territory.from_names().is_empty():
    print("empty")

not empty
empty


In [27]:
from itertools import chain

with open("../test.pickle", "rb") as file:
    epci_to_comm = pickle.load(file)


Territory.from_names(*tuple(chain(*epci_to_comm.values())))

Limersheim|Sarrageois|Vandenesse|Archelange|Neuviller-la-Roche|Héry|Palaja|Liesle|Beaufou|Saint-Brandan|Saint-Georges-de-Luzençon|Vallentigny|Isola|Nordhouse|Péreyres|Weyer|Petersbach|Bayecourt|Arrentières|Quenne|Saint-Andelain|Saint-Étienne-du-Valdonnez|Béard|La Grange|Aunay-en-Bazois|Villecomtal|Mariac|Chanéac|Villiers-Louis|Autigny-la-Tour|Champlemy|Mognéville|Martinet|Roches|Sombacour|Blessac|Vinneuf|Thillombois|Montgradail|Brousses-et-Villaret|Chauvoncourt|Nommay|Saint-Christophe-du-Bois|Schœnbourg|Domèvre-sur-Durbion|Spéracèdes|Montigny-lès-Arsures|Bartherans|Aubigné-sur-Layon|Sarry|Crevant|Meistratzheim|Mandeure|Soulaines-sur-Aubance|Plouédern|Bagneux-la-Fosse|Tréguidel|Rocles|Menetou-Salon|Val-d'Auzon|La Vernelle|Cézy|Saint-Divy|Argelès-sur-Mer|La Chapelle-Taillefert|Saint-Côme-d'Olt|Virey-sous-Bar|Autréville-Saint-Lambert|Macey|Viviers-le-Gras|Mende|Badevel|Saint-Jacut-de-la-Mer|Saint-Aubin-les-Forges|Gimouille|Pierrefontaine-les-Varans|Roquefort-sur-Soulzon|Cransac|Moulainvil

In [5]:
min(ter).partition_type

<Partition.DEP: 2>

In [8]:
ter.type

<Partition.DEP: 2>